In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msgn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest

In [2]:
data_products = pd.read_csv("products.csv")
data_user_dem = pd.read_csv("user_demographics.csv")
data_qa = pd.read_csv("qa.csv")
data_sup_def = pd.read_csv("supplier_defective_return.csv")
data_reviews = pd.read_csv("reviews.csv")
data_tra = pd.read_csv("transactions.csv")
data_sup_return = pd.read_csv("supplier_return.csv")
data_sup_disp = pd.read_csv("supplier_disputed_return.csv")

C:\Users\Salih\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data_products.isnull().sum()

product_id                0
product_content_id        0
product_variant_id        0
product_name              0
brand_id                  0
brand_name                0
gender_id              2452
gender_name               0
category_id               0
category_name             0
color_id               1527
color_name             1527
supplier_color_name       1
attributet_name           0
attribute_value           0
dtype: int64

In [4]:
data_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457567 entries, 0 to 1457566
Data columns (total 15 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   product_id           1457567 non-null  int64  
 1   product_content_id   1457567 non-null  int64  
 2   product_variant_id   1457567 non-null  int64  
 3   product_name         1457567 non-null  object 
 4   brand_id             1457567 non-null  int64  
 5   brand_name           1457567 non-null  object 
 6   gender_id            1455115 non-null  float64
 7   gender_name          1457567 non-null  object 
 8   category_id          1457567 non-null  int64  
 9   category_name        1457567 non-null  object 
 10  color_id             1456040 non-null  float64
 11  color_name           1456040 non-null  object 
 12  supplier_color_name  1457566 non-null  object 
 13  attributet_name      1457567 non-null  object 
 14  attribute_value      1457567 non-null  object 
dty

In [5]:
data_tra.isnull().sum()

order_date                      0
user_id                         0
is_elite_user                   0
supplier_id                     0
order_line_item_id              0
order_parent_id                 0
product_content_id              0
product_variant_id              0
original_price                  0
discounted_price                0
ship_cost                   36886
coupon_id                10433735
coupon_discount                 0
promotion_name            5653910
promotion_award_value     5653910
is_wallet_trx                   0
is_saved_card_trx               0
is_returned                421016
dtype: int64

In [6]:
data_tra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11193677 entries, 0 to 11193676
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   order_date             object 
 1   user_id                int64  
 2   is_elite_user          int64  
 3   supplier_id            int64  
 4   order_line_item_id     int64  
 5   order_parent_id        int64  
 6   product_content_id     int64  
 7   product_variant_id     int64  
 8   original_price         float64
 9   discounted_price       float64
 10  ship_cost              float64
 11  coupon_id              object 
 12  coupon_discount        float64
 13  promotion_name         object 
 14  promotion_award_value  float64
 15  is_wallet_trx          int64  
 16  is_saved_card_trx      bool   
 17  is_returned            float64
dtypes: bool(1), float64(6), int64(8), object(3)
memory usage: 1.4+ GB


In [7]:
#data_tra[data_tra["is_returned"]==1].sum()

In [8]:
data_sup_return[data_sup_return["supplier_id"]==434994]

,supplier_id,returnRate
20850,434994,0.15212


In [9]:
data_sup_return[data_sup_return["returnRate"]>0.7]

,supplier_id,returnRate
2939,363053,0.750000
2940,417873,1.000000
2941,141117,1.000000
2942,296346,1.000000
2943,348894,1.000000
2944,257591,1.000000
2945,169844,1.000000
2946,253325,1.000000
2947,346330,1.000000
2948,413221,1.000000


In [10]:
data_sup_disp.sort_values(by=["unresolvedclaim_percentage"], ascending=False)

,supplier_id,total_claim,unresolved_claim,unresolved_accepted_claim,unresolvedclaim_percentage
16556,140229,12,3,3,1.0
17235,357514,14,1,1,1.0
4347,280925,63,4,4,1.0
4346,141853,67,7,7,1.0
4345,114952,132,6,6,1.0
...,...,...,...,...,...
10573,409695,2,0,0,0.0
10574,259242,2,0,0,0.0
10575,184989,2,0,0,0.0
10576,189671,2,0,0,0.0


In [11]:
tra_prod_df = pd.merge(data_tra, data_products , on=['product_variant_id'])

In [12]:
tra_prod_df.isnull().sum()

order_date                      0
user_id                         0
is_elite_user                   0
supplier_id                     0
order_line_item_id              0
order_parent_id                 0
product_content_id_x            0
product_variant_id              0
original_price                  0
discounted_price                0
ship_cost                   36886
coupon_id                10433735
coupon_discount                 0
promotion_name            5653910
promotion_award_value     5653910
is_wallet_trx                   0
is_saved_card_trx               0
is_returned                421016
product_id                      0
product_content_id_y            0
product_name                    0
brand_id                        0
brand_name                      0
gender_id                    7223
gender_name                     0
category_id                     0
category_name                   0
color_id                     5217
color_name                   5217
supplier_color

In [13]:
tra_prod_user_df = pd.merge(tra_prod_df, data_user_dem , on=['user_id'])

In [14]:
tra_prod_user_df

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id_x,product_variant_id,original_price,discounted_price,...,category_id,category_name,color_id,color_name,supplier_color_name,attributet_name,attribute_value,birth_date,membership_date,gender
0,2021-05-01 00:00:01.761000+00:00,3941815,0,4662,901280147,595758968,95503167,159147197,79.90,51.94,...,604,T-Shirt,6998.0,Beyaz,Beyaz,BEDEN,XL,1979-05-24 00:00:00+00:00,2011-04-07 14:55:39.457000+00:00,Bayan
1,2021-05-01 00:00:01.761000+00:00,3941815,0,163,901280149,595758968,36313798,64622938,129.95,59.97,...,604,T-Shirt,7012.0,Yeşil,VR079,BEDEN,XL,1979-05-24 00:00:00+00:00,2011-04-07 14:55:39.457000+00:00,Bayan
2,2021-05-01 00:00:01.761000+00:00,3941815,0,157306,901280145,595758968,36888320,65612791,59.99,49.99,...,2949,Polo Yaka T-shirt,6999.0,Gri,GRİ_217,BEDEN,XL,1979-05-24 00:00:00+00:00,2011-04-07 14:55:39.457000+00:00,Bayan
3,2021-05-01 00:00:01.761000+00:00,3941815,0,163856,901280144,595758968,36888399,65613254,49.90,49.90,...,2949,Polo Yaka T-shirt,7006.0,Mor,MOR_11525,BEDEN,XL,1979-05-24 00:00:00+00:00,2011-04-07 14:55:39.457000+00:00,Bayan
4,2021-05-21 20:24:44.410000+00:00,3941815,0,112193,961445023,628941449,90524161,151332726,39.99,29.99,...,590,Bluz,7009.0,Siyah,Siyah,BEDEN,S,1979-05-24 00:00:00+00:00,2011-04-07 14:55:39.457000+00:00,Bayan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11193672,2021-07-28 11:12:38.107000+00:00,61273349,0,968,1101822442,718457351,94543481,198320577,139.99,118.99,...,1182,Elbise,7009.0,Siyah,Siyah,BEDEN,44,NaN,2021-03-17 22:34:12.553000+00:00,Bayan
11193673,2021-07-29 19:57:16.138000+00:00,71081533,0,968,1104684717,720153623,94543481,198320577,139.99,118.99,...,1182,Elbise,7009.0,Siyah,Siyah,BEDEN,44,NaN,2021-07-29 19:53:41.403000+00:00,UNKNOWN
11193674,2021-07-30 00:13:34.026000+00:00,40989877,0,968,1105071757,720405097,94543481,198320577,139.99,118.99,...,1182,Elbise,7009.0,Siyah,Siyah,BEDEN,44,NaN,2020-09-15 23:59:41.753000+00:00,Bayan
11193675,2021-07-31 14:26:52.679000+00:00,9611649,0,968,1107308286,721749486,94543481,198320577,139.99,118.99,...,1182,Elbise,7009.0,Siyah,Siyah,BEDEN,44,1984-06-10 00:00:00+00:00,2013-03-16 21:02:37.003000+00:00,Bayan


In [15]:
for col in tra_prod_user_df.columns:
    print('# of unique values in ', col, ': ', len(tra_prod_user_df[col].unique()))

# of unique values in  order_date :  4361897
# of unique values in  user_id :  706436
# of unique values in  is_elite_user :  2
# of unique values in  supplier_id :  22537
# of unique values in  order_line_item_id :  11193677
# of unique values in  order_parent_id :  4364619
# of unique values in  product_content_id_x :  705613
# of unique values in  product_variant_id :  1457567
# of unique values in  original_price :  26451
# of unique values in  discounted_price :  36719
# of unique values in  ship_cost :  4907
# of unique values in  coupon_id :  348778
# of unique values in  coupon_discount :  13079
# of unique values in  promotion_name :  3278
# of unique values in  promotion_award_value :  124
# of unique values in  is_wallet_trx :  2
# of unique values in  is_saved_card_trx :  2
# of unique values in  is_returned :  3
# of unique values in  product_id :  521401
# of unique values in  product_content_id_y :  705613
# of unique values in  product_name :  582378
# of unique values 

In [16]:
tra_prod_user_df["gender"].value_counts()

Bayan               9473062
Bay                 1538159
UNKNOWN              179896
kvkktalepsilindi       2560
Name: gender, dtype: int64

In [17]:
tra_prod_user_df["gender_name"].unique()

array(['Erkek', 'Kadın / Kız', 'Unisex', 'Unknown', 'Kadın'], dtype=object)

In [18]:
tra_prod_user_df["is_returned"].isna().sum()

421016

In [19]:
tra_prod_user_df.dropna(subset=["is_returned"], inplace=True)

In [20]:
tra_prod_user_df.isna().sum()

order_date                      0
user_id                         0
is_elite_user                   0
supplier_id                     0
order_line_item_id              0
order_parent_id                 0
product_content_id_x            0
product_variant_id              0
original_price                  0
discounted_price                0
ship_cost                   35102
coupon_id                10048720
coupon_discount                 0
promotion_name            5455583
promotion_award_value     5455583
is_wallet_trx                   0
is_saved_card_trx               0
is_returned                     0
product_id                      0
product_content_id_y            0
product_name                    0
brand_id                        0
brand_name                      0
gender_id                    6768
gender_name                     0
category_id                     0
category_name                   0
color_id                     4911
color_name                   4911
supplier_color

In [21]:
tra_prod_user_df["order_date"]

0           2021-05-01 00:00:01.761000+00:00
1           2021-05-01 00:00:01.761000+00:00
2           2021-05-01 00:00:01.761000+00:00
3           2021-05-01 00:00:01.761000+00:00
4           2021-05-21 20:24:44.410000+00:00
                          ...               
11193672    2021-07-28 11:12:38.107000+00:00
11193673    2021-07-29 19:57:16.138000+00:00
11193674    2021-07-30 00:13:34.026000+00:00
11193675    2021-07-31 14:26:52.679000+00:00
11193676    2021-07-31 21:59:05.290000+00:00
Name: order_date, Length: 10772661, dtype: object

In [22]:
tra_prod_user_df['order_weekday'] =  pd.to_datetime(tra_prod_user_df['order_date']).dt.weekday

In [ ]:
tra_prod_user_df['order_month'] = pd.to_datetime(tra_prod_user_df['order_date']).dt.month

In [ ]:
convert_hours = lambda x: datetime(1900, 1, 1, int(x)//100, int(x)%100)

In [ ]:
find_hour_difference = lambda x,y : (convert_hours(x)-convert_hours(y)).total_seconds()/60 if not pd.isna(y) else np.nan

In [ ]:
def pnr_workhour_interval(x):
    if 600 <= x < 2100:
        return 0
    else:
        return 1

In [ ]:
tra_prod_user_df['order_month'].unique()

In [ ]:
tra_prod_user_df["birth_date"] = tra_prod_user_df["birth_date"].fillna("0")

In [ ]:
tra_prod_user_df["age"] = tra_prod_user_df["birth_date"].apply(lambda x: 2021 - int(x.split('-')[0]))

In [ ]:
tra_prod_user_df["age"]

In [ ]:
tra_prod_user_df[(tra_prod_user_df["gender"]=="Bay") & (tra_prod_user_df["is_returned"]==1)]

In [ ]:
tra_prod_user_df.info()

In [ ]:
tra_prod_user_df['date'] = tra_prod_user_df["order_date"].apply(lambda x: int(''.join(x.split(' ')[1].split(':',2)[:2])))

In [ ]:
tra_prod_user_df['order_night_interval'] = tra_prod_user_df['date'].apply(pnr_workhour_interval)

In [ ]:
from datetime import datetime
from datetime import date

In [ ]:
tra_prod_user_df['membership_duration'] = tra_prod_user_df["membership_date"].apply(lambda x: (datetime.now() - datetime.strptime(str(x.split(' ')[0]), '%Y-%m-%d')).days)

In [ ]:
tra_prod_user_df.info()

In [ ]:
for i,j in zip(tra_prod_user_df["category_name"].value_counts(),tra_prod_user_df["category_name"].unique()):
    print(i,j)

In [ ]:
tra_prod_user_df[(tra_prod_user_df["category_name"]=="Fantezi Boxer")&(tra_prod_user_df["age"]==56)]

In [ ]:
tra_prod_user_df[tra_prod_user_df["user_id"]==33504151]

In [ ]:
for i in tra_prod_user_df[(tra_prod_user_df["birth_date"]== "1998-03-19 00:00:00+00:00")&(tra_prod_user_df["color_name"]=="Siyah")&(tra_prod_user_df["attribute_value"]=="M")]["order_date"]:
    print(i)

In [ ]:
tra_prod_user_df['color_name']

In [ ]:
tra_prod_user_sup_df = pd.merge(tra_prod_user_df, data_sup_return , on=['supplier_id'])

In [ ]:
tra_prod_user_sup_df = pd.merge(tra_prod_user_sup_df, data_sup_def , on=['supplier_id'])

In [ ]:
tra_prod_user_sup_df = pd.merge(tra_prod_user_sup_df, data_sup_disp , on=['supplier_id'])

In [ ]:
for i in sorted(tra_prod_user_sup_df['attribute_value'].unique()):
    print(i)

In [ ]:
data_reviews

In [ ]:
tra_prod_user_sup_df[tra_prod_user_sup_df['attribute_value']=="-4"]["category_name"]

In [ ]:
aynı kıyafetten geçmişte aldı mı almadı mı
son alışverişte aldı mı almadı mı
insanların commentlerinden iade nedenleri
yorumlardan bir şey çıkar mı
user id yi user dem 
content id lerle productları

In [4]:
test_data = pd.read_csv("test_data.csv")
test_user = test_data.id.str.split("|", expand=True)

In [8]:
test_user = pd.DataFrame(test_user).rename(columns = {0: 'user_id'})

In [7]:
test_content_id = test_data.id.str.split("|", expand=True)[1]
test_content_id = pd.DataFrame(test_content_id).rename(columns = {0: 'product_content_id'})
test_parent_id = test_data.id.str.split("|", expand=True)[2]
test_parent_id = pd.DataFrame(test_parent_id).rename(columns = {0: 'product_parent_id'})
test_user = test_user.astype(np.int64)
test_content_id = test_content_id.astype(np.int64)
test_parent_id = test_parent_id.astype(np.int64)
a = pd.merge(test_user, data_user_dem, how='left', on=["user_id"])
b = pd.merge(a, data_products, how='left', on=["product_content_id"])

KeyError: 'product_content_id'

In [3]:
c = pd.concat(a,b)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [40]:
a = pd.DataFrame([1,4,32,5,3,3,5])

In [41]:
a.drop([0], inplace=True)

In [42]:
a

,0
1,4
2,32
3,5
4,3
5,3
6,5


In [15]:
test_user_content_parent

,user_id,product_content_id,order_parent_id
0,18115,87073974,726348863
1,27588,118753298,723169934
2,29165,35699988,724122443
3,29969,5893187,722342150
4,35107,93897418,723421057
...,...,...,...
7466,71083899,100019570,728638514
7467,71106534,35624085,725372928
7468,71115350,94438742,725291348
7469,71115350,100017676,725291348


In [3]:
test_data = pd.read_csv("test_data.csv")
test_user_content_parent = test_data.id.str.split("|", expand=True)
test_user_content_parent = pd.DataFrame(test_user_content_parent).rename(columns = {0: 'user_id',1: 'product_content_id',2: 'order_parent_id' })
test_user_content_parent = test_user_content_parent.astype(np.int64)

In [4]:
at = test_user_content_parent.groupby(['order_parent_id', 'product_content_id'])['product_content_id'].count()
at

order_parent_id  product_content_id
722306383        35624085              1
722306627        100113358             1
722307246        4032898               1
722307624        94543481              1
722308875        35624085              1
                                      ..
729961384        118753298             1
729962375        68385037              1
729962660        35160248              1
729964390        72619411              1
729966025        91865490              1
Name: product_content_id, Length: 7471, dtype: int64

In [11]:
data_tra[data_tra["order_parent_id"]==729961384]

,order_date,user_id,is_elite_user,supplier_id,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned
11193279,2021-08-07 23:53:10.795000+00:00,14196527,1,968,1121259005,729961384,124142597,201213027,129.99,110.49,2.350,NaN,0.0,Trendyolmilla&Trendyol Man&Trendyol Modest Yen...,15.0,0,True,NaN
11193280,2021-08-07 23:53:10.795000+00:00,14196527,1,637,1121259012,729961384,127141948,212629963,39.99,27.99,4.440,NaN,0.0,NaN,NaN,0,True,NaN
11193281,2021-08-07 23:53:10.795000+00:00,14196527,1,330863,1121259008,729961384,118753298,191196254,113.95,91.16,4.700,NaN,0.0,Happiness İst. - Sepette %20 İndirim,20.0,0,True,NaN
11193282,2021-08-07 23:53:10.795000+00:00,14196527,1,319904,1121259016,729961384,84058125,141369939,250.00,133.68,5.665,NaN,0.0,NaN,NaN,0,True,NaN
11193283,2021-08-07 23:53:10.795000+00:00,14196527,1,968,1121259002,729961384,87081804,145734029,99.99,84.99,2.350,NaN,0.0,Trendyolmilla&Trendyol Man&Trendyol Modest Yen...,15.0,0,True,NaN
11193284,2021-08-07 23:53:10.795000+00:00,14196527,1,968,1121259006,729961384,48378347,85734527,69.99,59.49,2.350,NaN,0.0,Trendyolmilla&Trendyol Man&Trendyol Modest Yen...,15.0,0,True,NaN
11193285,2021-08-07 23:53:10.795000+00:00,14196527,1,968,1121259009,729961384,104384544,172385658,59.99,30.00,2.350,NaN,0.0,TRENDYOLMİLLA & TRENDYOL MAN & TRENDYOL MODEST...,50.0,0,True,NaN
11193286,2021-08-07 23:53:10.795000+00:00,14196527,1,1188,1121259010,729961384,99479624,165368446,39.99,29.99,4.440,NaN,0.0,NaN,NaN,0,True,NaN
11193287,2021-08-07 23:53:10.795000+00:00,14196527,1,319904,1121259003,729961384,84058392,141351817,250.00,133.51,5.665,NaN,0.0,NaN,NaN,0,True,NaN
11193288,2021-08-07 23:53:10.795000+00:00,14196527,1,330863,1121259013,729961384,126934595,212052242,114.95,91.96,4.700,NaN,0.0,Happiness İst. - Sepette %20 İndirim,20.0,0,True,NaN


In [19]:
data_tra["is_returned"].value_counts()

0.0    9402321
1.0    1370340
Name: is_returned, dtype: int64

In [5]:
test_user_content_parent[test_user_content_parent['order_parent_id']==726192589]

,user_id,product_content_id,order_parent_id
5520,26763343,93702528,726192589
5521,26763343,94176072,726192589
5522,26763343,94633274,726192589


In [26]:
at.sort_values()

order_parent_id  product_content_id
722306383        35624085              1
727583547        36361687              1
727582698        87073974              1
727581790        60111256              1
727579827        87474586              1
                                      ..
725197078        6857428               1
725195263        76716282              1
725194583        94639099              1
725206220        36899141              1
729966025        91865490              1
Name: product_content_id, Length: 7471, dtype: int64

In [16]:
df = pd.merge(test_user_content_parent, data_user_dem, on=['user_id'])
df = pd.merge(df, )

,user_id,product_content_id,order_parent_id,birth_date,membership_date,gender
0,18115,87073974,726348863,1991-07-10 00:00:00+00:00,2010-03-04 22:01:54.807000+00:00,Bayan
1,27588,118753298,723169934,1983-10-25 00:00:00+00:00,2010-03-18 21:03:56.257000+00:00,Bayan
2,29165,35699988,724122443,1987-04-24 00:00:00+00:00,2010-03-19 06:37:24.493000+00:00,Bayan
3,29969,5893187,722342150,1978-04-10 00:00:00+00:00,2010-03-19 07:12:01.027000+00:00,Bayan
4,35107,93897418,723421057,1980-06-27 00:00:00+00:00,2010-03-19 19:31:52.870000+00:00,Bayan
...,...,...,...,...,...,...
7466,71083899,100019570,728638514,NaN,2021-07-29 20:49:44.140000+00:00,UNKNOWN
7467,71106534,35624085,725372928,NaN,2021-07-30 13:37:41.387000+00:00,UNKNOWN
7468,71115350,94438742,725291348,NaN,2021-07-30 17:08:49.257000+00:00,Bayan
7469,71115350,100017676,725291348,NaN,2021-07-30 17:08:49.257000+00:00,Bayan


In [27]:
d = pd.merge(test_user_content_parent, data_tra, how='left', on=["user_id", "product_content_id", "order_parent_id"])

In [30]:
data_tra.shape

(11193677, 18)

In [25]:
d.drop_duplicates(subset=['product_content_id','order_parent_id'],keep='last', inplace=True)

In [26]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7471 entries, 0 to 7768
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                7471 non-null   int64  
 1   product_content_id     7471 non-null   int64  
 2   order_parent_id        7471 non-null   int64  
 3   order_date             7471 non-null   object 
 4   is_elite_user          7471 non-null   int64  
 5   supplier_id            7471 non-null   int64  
 6   order_line_item_id     7471 non-null   int64  
 7   product_variant_id     7471 non-null   int64  
 8   original_price         7471 non-null   float64
 9   discounted_price       7471 non-null   float64
 10  ship_cost              7460 non-null   float64
 11  coupon_id              598 non-null    object 
 12  coupon_discount        7471 non-null   float64
 13  promotion_name         4874 non-null   object 
 14  promotion_award_value  4874 non-null   float64
 15  is_w

In [20]:
data_products[data_products["product_content_id"]==77520098]

,product_id,product_content_id,product_variant_id,product_name,brand_id,brand_name,gender_id,gender_name,category_id,category_name,color_id,color_name,supplier_color_name,attributet_name,attribute_value
3,66208922,77520098,132019116,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,XS
876,66208922,77520098,132018754,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,XXS
928,66208922,77520098,132018812,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,XXL
1216,66208922,77520098,132019179,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,M
1387,66208922,77520098,132019094,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,XL
1883,66208922,77520098,132018904,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,S
2092,66208922,77520098,131993890,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,L


In [19]:
data_products

,product_id,product_content_id,product_variant_id,product_name,brand_id,brand_name,gender_id,gender_name,category_id,category_name,color_id,color_name,supplier_color_name,attributet_name,attribute_value
0,86747186,49156804,87148849,3'lü Müslin Örtü Seti - %100 Pamuk Bebek Müsli...,995186,Li Puna,NaN,Unknown,4979,Müslin Bez,NaN,NaN,UNKNOWN,BEDEN,Tek Ebat
1,37159399,44391553,78625630,Haşema Nehar Bakır Çizgili Boru Paça Siyah Tes...,35046,Remsa Mayo,4295.0,Kadın / Kız,2760,Tesettür Mayo,7005.0,Metalik,BAKIR,BEDEN,XL
2,83771873,96708156,161092495,Zincir Fantezi Bel Zinciri-0610,1074023,vonda wear,4295.0,Kadın / Kız,3098,Fantezi Aksesuar,6996.0,Altın,Gold,BEDEN,Tek Ebat
3,66208922,77520098,132019116,Kadın Haki Yeşili Renkli Cerrahi Takım,961534,Moodamedikal,NaN,Unknown,3914,İş Kıyafetleri,NaN,NaN,UNKNOWN,BEDEN,XS
4,68189697,79713829,135202630,Ibıza Uzun Tunik,1021561,Jadeandmate,4295.0,Kadın / Kız,1018,Plaj Elbisesi,6996.0,Altın,Multi,BEDEN,M/L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457562,105235826,119081308,191792203,Henna Dantelli Haki Bralet Takım,8173,Cansoy,4295.0,Kadın / Kız,2144,İç çamaşırı takımları,7015.0,Haki,Haki,BEDEN,75B
1457563,80081954,92558062,154329388,Askerlik Seti Acemi Asker 6'lı Temel Bedelli A...,977589,Silyon Askeri Giyim,4294.0,Erkek,2144,İç çamaşırı takımları,7015.0,Haki,Haki-2,BEDEN,M
1457564,80258129,45437018,80173430,Haki Dolgulu Destekli Balenli Dantel Bralet I...,963578,BESMOD,4295.0,Kadın / Kız,2144,İç çamaşırı takımları,7015.0,Haki,Haki,BEDEN,80B
1457565,80081954,93307594,155518766,3'lü Temel Askerlik Seti Acemi Asker - Bedelli...,977589,Silyon Askeri Giyim,4294.0,Erkek,2144,İç çamaşırı takımları,7015.0,Haki,Haki,BEDEN,L


In [25]:
a.head()

,user_id,product_content_id,order_parent_id,order_date,is_elite_user,supplier_id,order_line_item_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned
0,18115,87073974,726348863,2021-08-04 16:03:34.498000+00:00,1,968,1115248748,145716801,89.99,53.99,9.400000,NaN,0.0,NaN,NaN,1,False,NaN
1,27588,118753298,723169934,2021-08-01 22:32:11.468000+00:00,0,968,1109728166,191196258,113.95,91.16,1.044444,NaN,0.0,NaN,NaN,0,True,NaN
2,29165,35699988,724122443,2021-08-02 21:39:28.579000+00:00,1,1188,1111326869,63808285,119.99,83.99,8.880000,NaN,0.0,Defacto sepet 30 - Sepette %30 İndirim,30.0,0,False,NaN
3,29969,5893187,722342150,2021-08-01 01:07:16.583000+00:00,1,968,1108280452,19449310,59.99,50.99,4.700000,NaN,0.0,NaN,NaN,0,False,NaN
4,35107,93897418,723421057,2021-08-02 10:54:53.842000+00:00,1,968,1110143484,156456704,69.99,59.49,4.700000,NaN,0.0,Trendyolmilla&Trendyol Man&Trendyol Modest Yen...,15.0,0,False,NaN


In [26]:
test_user_content_parent

,user_id,product_content_id,order_parent_id
0,18115,87073974,726348863
1,27588,118753298,723169934
2,29165,35699988,724122443
3,29969,5893187,722342150
4,35107,93897418,723421057
...,...,...,...
7466,71083899,100019570,728638514
7467,71106534,35624085,725372928
7468,71115350,94438742,725291348
7469,71115350,100017676,725291348


In [ ]:
replace_for_errors= {'ADDS ABABA' : 'ADDIS ABABA',
                    'MOGADSHU' : 'MOGADISHU',
                    'SNOP' : 'SINOP',
                    'ABDJAN':'ABIDJAN',
                    'THESSALONK' : 'THESSALONIKI',
                    'SULEYMANAH' : 'SULEYMANIAH',
                    'HO CH MNH CTY (SAGON)' : 'HO CHI MINH CITY (SAIGON)',
                    'SHRAZ' : 'SHIRAZ',
                    'SHARM EL-SHEKH' : 'SHARM EL-SHEIKH',
                    'VALENCA' : 'VALENCIA',
                    'TURN(TORNO)' : 'TURIN(TORINO)',
                    'TEKRDAG' : 'TEKIRDAG',
                    'LEFKOA' : 'LEFKOSA',
                    'KLMANJARO' : 'KILIMANJARO',
                    'SRT' : 'SIIRT'
                    }

replace_for_errors = {k.lower(): v.lower() for k,v in replace_for_errors.items()}

In [ ]:
tra_prod_user_sup_df[["product_content_id_y", "product_content_id_x"]]

In [ ]:
df_v1['id'] = df_v1[["user_id","product_content_id","order_parent_id"]].apply(
    lambda x: '|'.join(x.astype(str)),
    axis=1)

In [ ]:
tra_prod_user_sup_df

In [ ]:
tra_prod_user_sup_df["user_id"]

In [ ]:
tra_prod_user_sup_df["brand_name"].value_counts()

In [ ]:
tra_prod_user_sup_df.isna().sum()

In [ ]:
label = ['is_returned']

categorical_str = [
'promotion_name',
'is_saved_card_trx',
'gender_name',
'category_name',
'color_name',
'supplier_color_name',
'attribute_value',
'gender',
'brand_name'
] 

categorical_int = [
'gender_id',
'category_id',
'color_id',
'order_weekday',
'order_month',
'brand_id',
]

numerical = [
'original_price',
'discounted_price',
'ship_cost',
'coupon_discount',
'age',
'membership_duration',
'total_claim',
'unresolved_claim',
'unresolved_accepted_claim',
'promotion_award_value'
]

passthrough = [
'is_elite_user',
'is_wallet_trx',
'order_night_interval',
'returnRate_x',
'returnRate_y',
'unresolvedclaim_percentage'

]

drops = [
'order_date',
'order_line_item_id',
'order_parent_id',
'product_content_id_x',
'product_variant_id',
'coupon_id',
'product_content_id_y',
'product_name',
'birth_date',
'membership_date',
'date',
'attributet_name']

In [ ]:
merged_df = tra_prod_user_sup_df[categorical_str+categorical_int+numerical+passthrough+label]

In [ ]:
merged_df.isnull().sum()

In [ ]:
matrix = merged_df.corr().abs()

# Create a mask
mask = np.triu(np.ones_like(matrix, dtype=bool))

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True)
plt.figure(figsize=(16, 12))
sns.heatmap(matrix[matrix>0.5], mask=mask, center=0, annot=True, fmt='.2f', square=True, cmap='crest')

plt.show();

In [ ]:
msgn.matrix(merged_df,color=(0.45, 0.15, 0.25))

In [ ]:
for i, col in enumerate(numerical):

    plt.figure(2*i)
    sns.countplot(x=col, data=merged_df)

    plt.figure(2*i+1)
    sns.boxplot(x=col, data=merged_df)


In [ ]:
merged_df[merged_df['age']>100]['age'].value_counts()

In [ ]:
merged_df['age'] = merged_df['age'].replace(2020, 0)

In [ ]:
merged_df['is_returned'].value_counts()

In [ ]:
def create_pipeline_rf():
    
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])
    

    categorical_str_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('one-hot', OneHotEncoder(handle_unknown='ignore'))])
    

    categorical_int_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value=-99)),
        ('one-hot', OneHotEncoder(handle_unknown='ignore'))])
    

    preprocessor = ColumnTransformer(
                                    transformers=[
                                                  ('num1', numeric_transformer, numerical),
                                                  
                                                  ('cat_str1', categorical_str_transformer, categorical_str),
                                                  
                                                  ('cat_int1', categorical_int_transformer, categorical_int),
                                                  ], 
                                    remainder='passthrough')

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('f_selector', SelectKBest(f_classif, k='all')),
                          ('classifier', RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0))])
    
    return pipeline

In [ ]:
y = merged_df['is_returned']
X = merged_df[categorical_str+categorical_int+numerical+passthrough]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=1)

pipe1 = create_pipeline_rf()

pipe1.fit(X_train, y_train)
preds = pipe1.predict(X_test)
print_metrics(preds, y_test, pipe1)

In [ ]:
sample = pd.read_csv("sample_df.csv")

In [ ]:
sample.info()

In [ ]:
sample['is_elite_user'] = sample['is_elite_user'].astype(np.int32)
sample['promotion_award_value'] = sample['promotion_award_value'].astype(np.float32)
sample['coupon_discount'] = sample['coupon_discount'].astype(np.float32)
sample['is_wallet_trx'] = sample['is_wallet_trx'].astype('category')
sample['ship_cost'] = sample['ship_cost'].astype(np.float32)
sample['brand_id'] = sample['brand_id'].astype(np.int32)
sample['category_id'] = sample['category_id'].astype(np.int32)
sample['gender_id'] = sample['gender_id'].astype(np.float32)
sample['color_id'] = sample['color_id'].astype(np.float32)
sample['gender'] = sample['gender'].astype('category')
sample['returnRate'] = sample['returnRate'].astype(np.float32)
sample['returnRate_def'] = sample['returnRate_def'].astype(np.float32)
sample['total_claim'] = sample['total_claim'].astype(np.float32)
sample['unresolvedclaim_percentage'] = sample['unresolvedclaim_percentage'].astype(np.float32)
sample['unresolved_percentage'] = sample['unresolved_percentage'].astype(np.float32)
sample['paid_amount'] = sample['paid_amount'].astype(np.float32)
sample['discount_ratio'] = sample['discount_ratio'].astype(np.float32)
sample['hour_interval'] = sample['hour_interval'].astype('category')
sample['diff_order_memdate'] = sample['diff_order_memdate'].astype(np.int32)
sample['order_weekday'] = sample['order_weekday'].astype('category')
sample['order_day'] = sample['order_day'].astype('category')
sample['order_week'] = sample['order_week'].astype('category')
sample['age'] = sample['age'].astype(np.int32)
sample['is_bday_close'] = sample['is_bday_close'].astype('category')
sample['attribute_value'] = sample['attribute_value'].astype('category')
sample['original_price'] = sample['original_price'].astype(np.float32)
sample['discounted_price'] = sample['discounted_price'].astype(np.float32)
sample['is_returned'] = sample['is_returned'].astype('category')

In [ ]:
sample['discounted_price'].value_counts()

In [ ]:
a = pd.read_csv("magaza_yorumlari_duygu_analizi.csv",encoding="utf-16")

In [ ]:
df['is_elite_user'] = df['is_elite_user'].astype(np.int32)
df['promotion_award_value'] = df['promotion_award_value'].astype(np.float32)
df['coupon_discount'] = df['coupon_discount'].astype(np.float32)
df['is_wallet_trx'] = df['is_wallet_trx'].astype('category')
df['ship_cost'] = df['ship_cost'].astype(np.float32)
df['brand_id'] = df['brand_id'].astype(np.int32)
df['category_id'] = df['category_id'].astype(np.int32)
df['gender_id'] = df['gender_id'].astype(np.float32)
df['color_id'] = df['color_id'].astype(np.float32)
df['gender'] = df['gender'].astype('category')
df['returnRate'] = df['returnRate'].astype(np.float32)
df['returnRate_def'] = df['returnRate_def'].astype(np.float32)
df['total_claim'] = df['total_claim'].astype(np.float32)
df['unresolvedclaim_percentage'] = df['unresolvedclaim_percentage'].astype(np.float32)
df['unresolved_percentage'] = df['unresolved_percentage'].astype(np.float32)
df['paid_amount'] = df['paid_amount'].astype(np.float32)
df['discount_ratio'] = df['discount_ratio'].astype(np.float32)
df['hour_interval'] = df['hour_interval'].astype('category')
df['diff_order_memdate'] = df['diff_order_memdate'].astype(np.int32)
df['order_weekday'] = df['order_weekday'].astype('category')
df['order_day'] = df['order_day'].astype('category')
df['order_week'] = df['order_week'].astype('category')
df['age'] = df['age'].astype(np.int32)
df['is_bday_close'] = df['is_bday_close'].astype('category')
df['attribute_value'] = df['attribute_value'].astype('category')
df['original_price'] = df['original_price'].astype(np.float32)
df['discounted_price'] = df['discounted_price'].astype(np.float32)
df['is_returned'] = df['is_returned'].astype('category')

In [ ]:
types = {
'is_elite_user':'int32',       
'promotion_award_value':'float32',
'coupon_discount':'float32',
'is_wallet_trx':'category',
'is_saved_card_trx':'category',
'ship_cost':'float32',
'brand_id':'int32',
'category_id':'int32',
'gender_id':'float32',
'color_id':'float32',
'gender':'category',
'returnRate':'float32',
'returnRate_def':'float32',
'total_claim':'float32',
'unresolvedclaim_percentage':'float32',
'unresolved_percentage':'float32',
'paid_amount':'float32',
'discount_ratio':'float32',
'hour_interval':'category',
'diff_order_memdate':'int32',
'order_weekday':'category',
'order_day':'category',
'order_week':'category',
'age':'int32',
'is_bday_close':'category',
'original_price':'float32',
'discounted_price':'float32',
'is_returned':'category',
'item_return_rate_x':'float32',
'item_return_rate_y':'float32',
'product_content_id':'float32',
'shifted_return_rate':'float32',
'zodiac':'category'}